# Live training | **2023-03-14 11am EST** | Supermarket Transaction Analysis in SQL | Martina Chiari

The checkout line is a familiar part of the retail experience. Any retailer needs to understand their checkout data to answer important questions like "do people spend more with card or cash?", "how many items do people buy at different times?", and "are self-service checkouts quicker than those with a cashier?". 

In this live training, you'll use SQL to analyze [transaction data from a Polish Supermarket](https://www.mdpi.com/2306-5729/4/2/67/htm) to answer financial and logistical questions.

A financial analysis of point of sale transactions can have many purposes. A non-exhaustive list is:
- increase sales by identifying what matters to the customer, is it promotions? on what items? is it opening hours? prices of particular items? and so on;
- increasing productivity of stores and operators by identifying factors that result in Lower processing time per item or basket size, accurate predictions for labour scheduling, adequate level of inventory, fewer scanning errors at checkout, and so on;
- predict sales in the short, medium and long term to inform decisions on investments in fixed assets and human resources, financing, go-to-market invesments, etc.

## Task 1: Exploring the data

A good first step in exploring a database is to select the first few rows from each table.

### Instructions

Select all columns and the first 100 rows from `pos_operator_logs` and `pos_transactions`.

In [1]:
SELECT *
FROM 'pos_operator_logs.csv'
LIMIT 100;

,id,Workstation_Group_ID,Workstation_ID,begin_date_time,operator_id
0,1,8,19,2019-02-13 05:37:55,269
1,2,8,18,2019-02-13 05:37:55,268
2,3,8,17,2019-02-13 05:38:43,267
3,4,1,4,2019-02-13 07:01:26,332
4,5,1,7,2019-02-13 07:01:57,10
...,...,...,...,...,...
95,96,1,20,2019-02-13 10:13:59,328
96,97,1,21,2019-02-13 10:17:43,338
97,98,1,20,2019-02-13 10:24:06,328
98,99,1,20,2019-02-13 10:24:24,328


In [2]:
SELECT *
FROM 'pos_transactions.csv'
LIMIT 100;

,id,WorkstationGroupID,begin_date_time,end_date_time,OperatorID,basket_size,t_cash,t_card,amount
0,1,1,2017-12-07 14:23:23,2017-12-07 14:24:36,101,23,True,False,112.71
1,2,1,2017-12-07 14:25:09,2017-12-07 14:27:00,101,29,True,False,54.76
2,3,1,2017-12-07 14:27:28,2017-12-07 14:27:48,101,3,True,False,14.77
3,4,1,2017-12-07 14:28:04,2017-12-07 14:28:29,101,12,True,False,37.88
4,5,1,2017-12-07 14:29:40,2017-12-07 14:30:32,101,7,True,False,115.34
...,...,...,...,...,...,...,...,...,...
95,96,1,2017-12-07 17:07:48,2017-12-07 17:08:39,101,9,True,False,32.54
96,97,1,2017-12-07 17:10:51,2017-12-07 17:11:33,101,9,True,False,26.59
97,98,1,2017-12-07 17:11:47,2017-12-07 17:12:37,101,20,True,False,104.82
98,99,1,2017-12-07 17:12:51,2017-12-07 17:13:58,101,12,False,True,67.62


## Task 2: Do more people make transactions by card or by cash?

Understanding how people pay is crucial for shop logistics like deciding what kind of checkout equipment to buy.

### Instructions

Count how many transactions were processed by cash (the case when `t_cash` is true) and by card  (the case when `t_card` is true). Use the `pos_transactions` table.

In [9]:
SELECT COUNT(*) AS by_cash, (SELECT COUNT(*) FROM 'pos_transactions.csv' WHERE t_card = 'True') AS by_card 
FROM 'pos_transactions.csv'
WHERE t_cash = 'True';

,by_cash,by_card
0,84487,78246


In [12]:
SELECT
COUNT(CASE WHEN t_cash THEN 1 END) AS by_cash,
COUNT(CASE WHEN t_card THEN 1 END) AS by_card
FROM 'pos_transactions.csv';

,by_cash,by_card
0,84487,78246


## Task 3: Do people spend more per transaction when using cash or card?

Similarly, knowing how much people spend per transaction using different payment methods is helpful for deciding on security arrangments, and marketing to encourage shoppers to use different payment methods.

Find the average per transaction type when a transaction is only card, or only cash.

### Instructions

Calculate the mean (average) amount spent per transaction for cash and for card.

In [20]:
SELECT ROUND(AVG(amount),2) AS cash_avg, (SELECT ROUND(AVG(amount),2) FROM 'pos_transactions.csv' WHERE t_card = 'True' AND t_cash = 'False') AS card_avg
FROM 'pos_transactions.csv' 
WHERE t_cash = 'True' AND t_card = 'False'

,cash_avg,card_avg
0,58.72,86.84


In [21]:
SELECT
ROUND(AVG(CASE WHEN t_cash AND NOT t_card THEN amount END),2) AS cash_avg,
ROUND(AVG(CASE WHEN t_card AND NOT t_cash THEN amount END),2) AS card_avg
FROM 'pos_transactions.csv';

,cash_avg,card_avg
0,58.72,86.84


## Task 4: Exploring Sunday Trading

Regulations were introduced in Poland in 2018 banned shopping on some Sundays. It was generally two Sundays per month in 2018 and three Sundays per month in 2019. Supermarkets have reacted by extending the working hours on Fridays and Saturdays, while customers had to adapt to the changes in opening hours. **The dataset contains two working Sundays (24 February, 31 March) and two non-working Sundays (17 February, 7 April).**


### Instructions

Get the week number of the dates where trading occured on a Sunday. 

- Find the week number of `end_date_time` from `pos_transactions` for these Sundays: 17 February, 24 February,31 March and 7 April 2019.
- Order the dates from latest to earliest.

In [4]:
SELECT
DATE_PART('week', end_date_time::DATE) AS week_num,
end_date_time::DATE AS end_date
FROM 'pos_transactions.csv'
WHERE EXTRACT(YEAR FROM end_date_time) >= 2019 
      AND end_date IN ('2019-02-17', '2019-02-24', '2019-03-31', '2019-04-07')
      GROUP BY end_date;

,week_num,end_date
0,8,2019-02-24
1,13,2019-03-31


The last two dates in my query do not appear and rightly so, because they were non-working Sundays. 

So the Weeks with with working Sundays are 8 and 13 ("WS"). And the weeks with non-working Sundays are 7 and 14 ("NWS"). 

Let's explore the data further to see if I have the full week data. 

## Task 5: Analysis of daily trends

Understanding which days are busiest is important for deciding how many staff to schedule shifts for, as well as helping to plan financially.

### Instructions

- Select all transactions of 2019 and find out the daily sales amount, the daily average amount per transaction and the median average amount per transaction. 
- group by week

Do we see different trends on different days of the week? 

In [5]:
SELECT
DATE_PART('week', end_date_time::DATE) AS week_num,
end_date_time::DATE AS end_date,
COUNT (id) AS total_transactions,
SUM(amount) AS total_sales,
AVG(amount) AS avg_sale_amount
FROM 'pos_transactions.csv'
WHERE EXTRACT(YEAR FROM end_date_time) >= '2019'
GROUP BY end_date
ORDER BY week_num;

,week_num,end_date,total_transactions,total_sales,avg_sale_amount
0,7,2019-02-13,3163,189112.49,59.788963
1,7,2019-02-14,4404,302242.45,68.629076
2,7,2019-02-15,4031,319112.44,79.164584
3,7,2019-02-16,4669,405580.79,86.866736
4,8,2019-02-23,4367,369412.97,84.591933
5,8,2019-02-18,3270,193133.29,59.062168
6,8,2019-02-19,3025,170242.42,56.278486
7,8,2019-02-21,4368,298612.75,68.363725
8,8,2019-02-22,4411,330428.78,74.910175
9,8,2019-02-24,2258,143025.41,63.341634


Build on your analysis adding the average and median `basket_size` per day in 2019. 



In [6]:
SELECT
DATE_PART('week', end_date_time::DATE) AS week_num,
end_date_time::DATE AS end_date,
COUNT(id) AS total_transactions,
SUM(amount) AS total_sales,
AVG(amount) AS avg_sale_amount,
PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY amount) AS median_sale_amount,
AVG(basket_size) AS avg_basket_size,
PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY basket_size) AS median_basket_size
FROM 'pos_transactions.csv'
WHERE EXTRACT(YEAR FROM end_date_time) >= '2019'
GROUP BY week_num, end_date
ORDER BY week_num;

,week_num,end_date,total_transactions,total_sales,avg_sale_amount,median_sale_amount,avg_basket_size,median_basket_size
0,7,2019-02-13,3163,189112.49,59.788963,37.580,12.915903,9.0
1,7,2019-02-14,4404,302242.45,68.629076,46.285,14.172116,10.0
2,7,2019-02-15,4031,319112.44,79.164584,50.900,17.184818,12.0
3,7,2019-02-16,4669,405580.79,86.866736,57.600,19.733348,14.0
4,8,2019-02-23,4367,369412.97,84.591933,56.370,19.813144,14.0
5,8,2019-02-18,3270,193133.29,59.062168,37.420,13.810092,10.0
6,8,2019-02-19,3025,170242.42,56.278486,37.530,13.432727,9.0
7,8,2019-02-21,4368,298612.75,68.363725,45.865,16.488324,12.0
8,8,2019-02-22,4411,330428.78,74.910175,49.060,17.736568,12.0
9,8,2019-02-24,2258,143025.41,63.341634,37.660,15.169176,10.0


## Task 6: Should the supermarket open on Sundays?

The two Sundays I see are significantly lower in volume of transactions than all other days. 
Also, Thursdays, Fridays and Saturdays are approximately 25% higher in number of transactions and 40% higher in sales revenue than Mondays, Tuesdays and Wednesdays.

Does it make sense to open stores on Sundays? 

The full week data is available only for week 8 (WS) and week 14 (NWS). Let's compare these two.


### Instructions

Calculate the total weekly sales amount for week 8 and 14 in 2019.

In [3]:
SELECT
DATE_PART('week', end_date_time::DATE) AS week_num,
ROUND(SUM(amount),2) AS total_sales
FROM 'pos_transactions.csv'
WHERE week_num IN (8,14)
GROUP BY week_num
ORDER BY week_num;

,week_num,total_sales
0,8,1686485.72
1,14,1690989.87


In [5]:
SELECT 
DATE_PART('week', end_date_time::DATE) as week_num,
ROUND(SUM(amount),2) AS total_sales
FROM 'pos_transactions.csv'

WHERE EXTRACT(YEAR FROM end_date_time) >= '2019' AND week_num IN ('8','14')

GROUP BY week_num

,week_num,total_sales
0,8,1686485.72
1,14,1690989.87


## Task 7: How much does labor cost?

Week 8 (WS) revenue is PLN1.7M and also week 14 (NWS) has PLN1.7M. It may be possible that customers go shopping before Sunday knowing that the store will be closed on that day. 

Assuming that the competition is going to also notice this trend and close on Sundays, and assuming that our stores can rearrange the workforce shifts to 6 days a week cutting the Sunday shifts **there may be possible savings from closing the stores on Sundays with no impact on revenue**. 

I start quantifying the savings with labour costs, how many operators were working on working Sundays? 

### Instructions

Find out how many operators were working on each day in 2019 by counting `operator_id `working on each `being_date_time`  

In [6]:
SELECT 
(begin_date_time::DATE) as working_day,
COUNT(DISTINCT(operator_id)),
DATE_PART('week',(begin_date_time::DATE)) as week_num

FROM 'pos_operator_logs.csv'

WHERE EXTRACT(YEAR FROM begin_date_time) >= '2019'

GROUP BY working_day

,working_day,count(DISTINCT operator_id),week_num
0,2019-02-13,20,7
1,2019-02-14,26,7
2,2019-02-15,24,7
3,2019-02-16,28,7
4,2019-02-18,20,8
5,2019-02-19,21,8
6,2019-02-20,19,8
7,2019-02-21,27,8
8,2019-02-22,26,8
9,2019-02-23,25,8


The stores had 18 employees working on Sunday 24 February and 20 employees working on Sunday 31 March. 
I would then look into the hours per shift and the cost per hour to calcute the average savings per Sunday, as well as estimate all other costs of keeping stores open and operational (e.g. energy costs). On the other hand stores may have additional costs for not opening one day, for example for short shelf life goods that may go unsold. 